# 第五章 一等函数

## 5.1 把函数视作对象

In [1]:
def factorial(n):
    '''returns n!'''
    return 1 if n < 2 else factorial(n-1) * n
factorial(42), factorial.__doc__, type(factorial)

(1405006117752879898543142606244511569936384000000000, 'returns n!', function)

示例 5-2 通过别称使用函数, 再把函数作为参数传递

In [2]:
fact = factorial
fact, fact(5), list(map(fact, range(10))) 

(<function __main__.factorial(n)>,
 120,
 [1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880])

## 5.2 高阶函数

> 接受函数作为参数, 或者把函数作为结果返回的函数称为高阶函数

示例 5-3 根据单词长度给一个列表序列排序

In [3]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len), min(fruits, key=len) 

(['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry'], 'fig')

示例 5-4 根据反向拼写给一个单词列表排序

In [4]:
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

示例 5-6 使用 reduce 和 sum 求和

In [5]:
from functools import reduce
from operator import add
reduce(add, range(100)), sum(range(100))

(4950, 4950)

In [6]:
iterable = [True, False, True]
all(iterable), any(iterable)

(False, True)

## 5.3 匿名函数

In [7]:
lst = [[1, 2], [5, 2], [0, 1], [-2, 2]]
sorted(lst, key=lambda x: x[0])

[[-2, 2], [0, 1], [1, 2], [5, 2]]

In [8]:
sorted(lst, key=lambda x: sum(x))

[[-2, 2], [0, 1], [1, 2], [5, 2]]

## 5.4 可调用对象

* 用户定义函数: 使用`def`或`lambda`表达式创建
* 内置函数: 使用 `CPython` 实现的函数, 如 `len` 或 `time.strftime`
* 内置方法: 使用C语言实现的方法, 如 `dict.get`
* 方法: 在类的定义体中定义的函数
* 类: 调用类时会运行类的`__new__`方法创建一个实例, 然后运行`__init__ `方法, 初始化实例, 最后把实例返回给调用方
* 类的实例: 通过 `__call__` 方法
* 生成器函数: 使用 `yield` 关键字的函数或方法, 返回的是生成器对象

> callable可以判断对象是否可以调用

In [9]:
[callable(obj) for obj in (abs, str, 12)]

[True, True, False]

## 5.5 用户自定义的可调用类型

In [10]:
import random

class BingoCage:

    def __init__(self, items):
        self._items = list(items)  # <1>
        random.shuffle(self._items)  # <2>

    def pick(self):  # <3>
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')  # <4>

    def __call__(self):  # <5>
        return self.pick()

In [11]:
bingo = BingoCage(range(10))
bingo(), bingo(), bingo()

(3, 8, 5)

In [12]:
bingo._items

[7, 9, 4, 6, 0, 2, 1]

In [13]:
callable(bingo)

True

## 5.6 函数内省

In [14]:
from pprint import pprint
def format_list(lst: list, n: int = 5) -> list:
    return [lst[i:i + n] for i in range(0, len(lst), n)]

函数的所有属性

In [15]:
pprint(format_list(dir(factorial)))

[['__annotations__', '__builtins__', '__call__', '__class__', '__closure__'],
 ['__code__', '__defaults__', '__delattr__', '__dict__', '__dir__'],
 ['__doc__', '__eq__', '__format__', '__ge__', '__get__'],
 ['__getattribute__', '__getstate__', '__globals__', '__gt__', '__hash__'],
 ['__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__'],
 ['__module__', '__name__', '__ne__', '__new__', '__qualname__'],
 ['__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__'],
 ['__str__', '__subclasshook__']]


`__dict__`属性

示例 5-9 列出常规对象没有而函数有的属性

In [24]:
from typing import Tuple
class C: pass
obj = C()
def func(n: int = 10) -> Tuple[int]: 
    '''This is fun''' 
    return n - 1
pprint(format_list(list(set(dir(func)) - set(dir(obj)))))

[['__qualname__', '__get__', '__name__', '__globals__', '__builtins__'],
 ['__code__', '__closure__', '__defaults__', '__call__', '__kwdefaults__'],
 ['__annotations__']]


In [25]:
func.__annotations__

{'n': int, 'return': typing.Tuple[int]}

In [26]:
f = func
f.__name__

'func'

In [27]:
f.__call__(1)

0

In [28]:
func.__code__   #  编译成字节码的函数元数据和函数定义体

<code object func at 0x000002C1DA375D70, file "C:\Users\19145\AppData\Local\Temp\ipykernel_32884\1928376850.py", line 4>

In [29]:
func.__defaults__   # 形式函数的默认值

(10,)

In [30]:
func.__get__    # 实现只读描述符协议

<method-wrapper '__get__' of function object at 0x000002C1DA1D7880>

In [31]:
func.__globals__

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  "def factorial(n):\n    '''returns n!'''\n    return 1 if n < 2 else factorial(n-1) * n\nfactorial(42), factorial.__doc__, type(factorial)",
  'fact = factorial\nfact, fact(5), list(map(fact, range(10))) ',
  "fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']\nsorted(fruits, key=len), min(fruits, key=len) ",
  'sorted(fruits, key=lambda word: word[::-1])',
  'from functools import reduce\nfrom operator import add\nreduce(add, range(100)), sum(range(100))',
  'iterable = [True, False, True]\nall(iterable), any(iterable)',
  'lst = [[1, 2], [5, 2], [0, 1], [-2, 2]]\nsorted(lst, key=lambda x: x[0])',
  'sorted(lst, key=lambda x: sum(x))',
  '[callable(obj) for obj in (abs, str, 12)]',
  "impo

In [32]:
func.__kwdefaults__

In [33]:
func.__qualname__

'func'

## 5.7 从定位参数到仅限关键字参数

示例 5-10 tag 函数用于生成 HTML 标签；使用名为 cls 的关键字参数传入“class”属性，这是一种变通方法，因为“ class”是 Python的关键字

In [43]:
from ipywidgets import HTML

def tag(name, *content, cls=None, **attrs):
    """Generate one or more HTML tags"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                           for attr, value
                           in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                         (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [44]:
HTML(tag('br'))

HTML(value='<br />')

In [42]:
HTML(tag('div',"Hello", "World"))

HTML(value='<div>Hello</div>\n<div>WOrld</div>')

In [49]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
HTML(tag(**my_tag))

HTML(value='<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />')

 使用 * 实现仅限关键字参数

In [50]:
def f(a, *, b):
    return a + b

In [52]:
try:
    f(1, 2)
except Exception as e:
    print(e)

f() takes 1 positional argument but 2 were given


In [54]:
f(1, b=2)  

3

## 5.9 函数注解

In [55]:
func.__annotations__

{'n': int, 'return': typing.Tuple[int]}

## 5.10 支持函数式编程的包

### 5.10.1 `operator`模块

示例 5-21 使用 reduce 函数和一个匿名函数计算阶乘

In [57]:
from functools import reduce
def fact(n):
    return reduce(lambda a, b: a * b, range(1, n+1))
fact(5)

120

示例 5-22 使用 reduce 和 operator.mul 函数计算阶乘

In [58]:
from functools import reduce
from operator import mul
fact = lambda n: reduce(mul, range(1, n+1))
fact(5)

120

示例 5-23 演示使用 itemgetter 排序一个元组列表

In [59]:
from operator import itemgetter
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),   # <1>
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

In [63]:
for city in sorted(metro_areas, key=itemgetter(2)): print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))


In [65]:
for city in metro_areas:
    print(itemgetter(1, 0)(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


### 5.10.2 使用functools.partial 冻结参数

In [66]:
from operator import mul
from functools import partial

In [67]:
triple = partial(mul, 3)
triple(4)

12

In [68]:
[triple(i) for i in range(10)]

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

In [70]:
def student(name, *, age, gender):
    return f"{name} | {age} | {gender}"
student("Rookie", age=18, gender="男")

'Rookie | 18 | 男'

In [71]:
stu = partial(student, age=100, gender='male')
stu('Rookie')

'Rookie | 100 | male'

## 5.11 本章小结

<img src="./images/第五章总结.jpg" width="70%">